Script to extract 4 words from responses. While participants were asked to provide 4 words, sometimes they also provided a description. 

In [2]:
from openai import OpenAI
client = OpenAI(api_key="Your Key Here")

In [4]:
import pandas as pd
import os


In [ ]:
df = pd.read_csv("path_to_csv")
df.head()

In [16]:
def create_deduplicate_prompt(sentence, response):
    prompt = f"""You are scoring a response from a large language model that is simulating a person.
However, the model is not very well trained and the response may be garbled or repeat sections.
Your job is to extract the portion of the model’s response that seems to be relevant and ignore exact repetitions.
If the response contains rephrasings of the same idea, keep both rephrasings.
Keep as much of the initial response as possible.
In some cases, the model may complete the sentence, then generate one or more additional prompt(s) and completion(s).
Do not include the additional prompt(s) or completion(s).
The model was asked to complete the following sentence: {sentence}
The model responded: {response}
Please respond only with the pared down version of the model response.

"""
    return prompt

In [ ]:
prompts = [create_deduplicate_prompt(df['sentence'][i], df['response'][i]) for i in range(len(df))]

In [26]:
#Sanity Check
responses = []
for i in range(5):
    completion = client.chat.completions.create(model="gpt-4o-mini",
              messages=[
                {"role": "system", "content": "You are an evaluator helping to evaluate language model outputs."},
                {"role": "user", "content": prompts[i]}])
    response =completion.choices[0].message.content.strip()
    responses.append(response)

In [39]:
%%time
folder = "./path/to/file/"
for file in os.listdir(folder):
    if file.endswith('end_of_file.csv'):
        #if"gemma-2-9b" not in file: #Gemma-2-9b failed for us, so we don't want to pay openAI to evaluate null responses 
        if True:
            df = pd.read_csv(folder+file)
            prompts = []
            responses = []
            for i in range(len(df)):
                prompt = create_pp_prompt(df['prompt'][i],df['response'][i])
                prompts.append(prompt)
                completion = client.chat.completions.create(model="gpt-4o",
                          messages=[
                            {"role": "system", "content": "You are an evaluator helping to evaluate language model outputs."},
                            {"role": "user", "content": prompts[i]}])
                response =completion.choices[0].message.content.strip()
                responses.append(response)
            new_df = pd.DataFrame(responses, columns =['refusal', 'word1', 'word2', 'word3', 'word4']) 
            df_combined = pd.concat([df, new_df], axis=1)
            new_df.to_csv(folder+'deduplicated/'+file, index=False)
        
        

CPU times: total: 3.16 s
Wall time: 11min 55s
